# **Z3 Library **

In [ ]:
from z3 import *

# **Importing data**

In [ ]:
instance = 24
inst_str = 'ins-' + str(instance) + '.txt'
dir = 'Instances/'
with open(dir + inst_str) as f:
    data = f.readlines()
data = [x.strip() for x in data]
data = [x for x in data if x]
width = int(data[0])
num_circuits = int(data[1])
circuitX = []
circuitY = []
for i in range(2, len(data)):
    circuitX.append(int(data[i].split(' ')[0]))
    circuitY.append(int(data[i].split(' ')[1]))

# **Variables**

In [ ]:
X = [z3.Int('X_%s' % i) for i in range(num_circuits)]
Y = [z3.Int('Y_%s' % i) for i in range(num_circuits)]
Height = Int('Height')

# **Main Problem constraints**

In [ ]:
min_height = max(circuitY)
max_height = sum(circuitY)
ext = []
ext.append(And(Height>=min_height,Height<=max_height))

In [ ]:
domain = [ And(X[i] >= 0, X[i] < width, Y[i] >= 0, Y[i] < Height)
             for i in range(num_circuits)] 

In [ ]:
circuit_dim = [And(circuitX[i] + X[i] <= width, circuitY[i] + Y[i]<=Height)for i in range(num_circuits) ]

In [ ]:
non_overlap = []
for i in range(num_circuits):
    for j in range(num_circuits):
        if (i<j):
            non_overlap.append(Or(X[i] + circuitX[i]  <= X[j], X[j] + circuitX[j] <= X[i], Y[i] + circuitY[i] <= Y[j], Y[j] + circuitY[j] <= Y[i]))

In [ ]:
implied = []
for i in range(width):
    for j in range(num_circuits):
        implied.append(Sum([If(And(X[j] <= i, i < X[j] + circuitX[j]), circuitY[j],0) for j in range(num_circuits)]) <= Height)

for i in range(num_circuits):
    for j in range(num_circuits):
        implied.append(Sum([If(And(Y[j] <= i, i < Y[j] + circuitY[j]), circuitX[j],0) for j in range(num_circuits)]) <= width)

In [ ]:
sym=[]
for i in range(num_circuits):
    for j in range(num_circuits):
        if (i<j):
            sym.append(Or(Not(And(circuitX[i] == circuitX[j], circuitY[i] == circuitY[j])), If(X[i] == X[j], Y[i] >= Y[j], X[i] > X[j])))

In [ ]:
constraints =  ext+domain+circuit_dim+non_overlap+implied+sym

# **Objective Function**

In [ ]:
s = Optimize()
s.add(constraints)
s.minimize(Height)

# **Output**

In [ ]:
inst_str = 'out-' + str(instance) + '.txt'
dir = 'Output/'
o = open(dir + inst_str, 'w')

In [ ]:
if s.check() == sat:
    m = s.model()
    print("{} {}".format(width, str(m[Height])))
    o.write(("{} {}".format(width, str(m[Height]))) + '\n')
    print("{}".format(num_circuits))
    o.write(("{}".format(num_circuits)) + '\n')
    for i in range(num_circuits):
        print("{:<1} {:<3} {:<1} {:<2}".format(circuitX[i], circuitY[i], str(m[X[i]]), str(m[Y[i]])))
        o.write(("{:<1} {:<3} {:<1} {:<2}".format(circuitX[i], circuitY[i], str(m[X[i]]), str(m[Y[i]])))+'\n')
    o.close()
else: 
    print("Failed")

31 31
19
3 3   3 17
3 4   3 0 
3 5   3 12
3 6   25 0 
3 7   28 24
3 8   3 4 
3 9   0 0 
3 10  28 14
3 11  3 20
3 12  25 19
3 13  25 6 
3 14  28 0 
3 22  0 9 
3 31  6 0 
4 3   9 15
4 7   9 0 
4 8   9 7 
4 13  9 18
12 31  13 0 
